In [ ]:
# Energy Forecasting with GRU - Jupyter Notebook Version

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from google.colab import files
import psycopg2



In [ ]:
host = 'jupyter.dreadcode.dev'
port = 5432
database = 'postgres'
user = 'postgres'
password = '5~;X*hLkvdf>xa*T'
# Configuración de la conexión
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password
)

# Crear un cursor
cur = conn.cursor()

# Ejecutar consulta para obtener datos de la vista
conn.rollback()  # Esto limpia el estado de transacción fallida. Sino una vez falle una consulta se va a quedar pegado.

cur.execute('SELECT "Date", "Consumo" FROM tablasxm_dev.demareal;')

datos = cur.fetchall()

column_names = [desc[0] for desc in cur.description]
df = pd.DataFrame(datos, columns=column_names)
df.head()

In [ ]:
'''
df['Date'] = pd.to_datetime(df['Date'],format='%d/%m/%Y %H:%M')
#df['Date'] = pd.to_datetime(df['Date'],format='%d/%m/%Y')
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Hour'] = df['Date'].dt.hour
'''

features = ['Hour','DayOfWeek','Day','Month','Year']

X = df[features]
Y = df[['Consumo']]

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_x.fit_transform(X)
y_scaled = scaler_y.fit_transform(Y)

sequence_length = 24

class EnergyDataset(Dataset):
    def __init__(self, x, y, seq_len):
        self.x = []
        self.y = []
        for i in range(len(x) - seq_len):
            self.x.append(x[i:i+seq_len])
            self.y.append(y[i+seq_len])
        self.x = torch.tensor(self.x, dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Crear datasets
dataset = EnergyDataset(X_scaled, y_scaled, sequence_length)
train_size = int(0.7 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
# 🧠 4. Modelo GRU
class EnergyGRUModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=64, num_layers=2, output_size=1, dropout=0.2):
        super(EnergyGRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 64).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

model_gru = EnergyGRUModel()
criterion_gru = nn.MSELoss()
optimizer_gru = torch.optim.Adam(model_gru.parameters(), lr=0.001)

'''
# 🔁 5. Entrenamiento
epochs = 10
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for x_batch, y_batch in train_loader:
        output = model(x_batch)
        loss = criterion(output, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}")
'''
# 🔁 5. Entrenamiento
epochs = 10
train_losses_gru = []
test_losses_gru = []
train_maes_gru = []
test_maes_gru = []

for epoch in range(epochs):
    model_gru.train()
    epoch_loss = 0
    train_predictions = []
    train_actuals = []
    for x_batch, y_batch in train_loader:
        output = model_gru(x_batch)
        loss = criterion_gru(output, y_batch)
        optimizer_gru.zero_grad()
        loss.backward()
        optimizer_gru.step()
        epoch_loss += loss.item()
        train_predictions.append(output.detach().numpy())
        train_actuals.append(y_batch.detach().numpy())

    train_losses_gru.append(epoch_loss/len(train_loader))
    train_predictions = scaler_y.inverse_transform(np.vstack(train_predictions))
    train_actuals = scaler_y.inverse_transform(np.vstack(train_actuals))
    train_mae = mean_absolute_error(train_actuals, train_predictions)
    train_maes_gru.append(train_mae)

    model_gru.eval()
    test_loss = 0
    test_predictions = []
    test_actuals = []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            output = model_gru(x_batch)
            loss = criterion_gru(output, y_batch)
            test_loss += loss.item()
            test_predictions.append(output.numpy())
            test_actuals.append(y_batch.numpy())

    test_losses_gru.append(test_loss/len(test_loader))
    test_predictions = scaler_y.inverse_transform(np.vstack(test_predictions))
    test_actuals = scaler_y.inverse_transform(np.vstack(test_actuals))
    test_mae = mean_absolute_error(test_actuals, test_predictions)
    test_maes_gru.append(test_mae)


    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses_gru[-1]:.4f}, Test Loss: {test_losses_gru[-1]:.4f}, Train MAE: {train_maes_gru[-1]:.4f}, Test MAE: {test_maes_gru[-1]:.4f}")

# Plotting the training and validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_losses_gru, label="Training Loss")
plt.plot(test_losses_gru, label="Validation Loss")
plt.title("Training and Validation Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.grid(True)

# Plotting the training and validation MAE
plt.subplot(1, 2, 2)
plt.plot(train_maes_gru, label="Training MAE")
plt.plot(test_maes_gru, label="Validation MAE")
plt.title("Training and Validation MAE per Epoch")
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 📊 6. Evaluación
model_gru.eval()
preds_gru, actuals_gru = [], []

with torch.no_grad():
    for x_batch, y_batch in test_loader:
        output = model_gru(x_batch)
        preds_gru.append(output.numpy())
        actuals_gru.append(y_batch.numpy())

preds_gru = scaler_y.inverse_transform(np.vstack(preds_gru))
actuals_gru = scaler_y.inverse_transform(np.vstack(actuals_gru))

# 📈 7. Visualización
plt.figure(figsize=(12, 4))
plt.plot(actuals_gru[:300], label="Real")
plt.plot(preds_gru[:300], label="Predicho")
plt.title("Consumo energético - Real vs Predicho")
plt.xlabel("Tiempos")
plt.ylabel("kWh")
plt.legend()
plt.show()

# Calcular métricas
mae_gru = mean_absolute_error(actuals_gru, preds_gru)
mse_gru = mean_squared_error(actuals_gru, preds_gru)
r2_gru = r2_score(actuals_gru, preds_gru)

print(f"MAE: {mae_gru:.4f}")
print(f"MSE: {mse_gru:.4f}")
print(f"R2 Score: {r2_gru:.4f}")

In [ ]:
# 🧠 4. Modelo LSTM
class EnergyLSTMModel(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, num_layers=2, output_size=1, dropout=0.2):
        super(EnergyLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 64).to(x.device)
        c0 = torch.zeros(2, x.size(0), 64).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model_gru = EnergyLSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model_lstm = EnergyLSTMModel()
criterion_lstm = nn.MSELoss()
optimizer_lstm = torch.optim.Adam(model_lstm.parameters(), lr=0.001)

'''# 🔁 5. Entrenamiento
epochs = 10
for epoch in range(epochs):
    model_lstm.train()
    epoch_loss = 0
    for x_batch, y_batch in train_loader:
        output = model_lstm(x_batch)
        loss = criterion_lstm(output, y_batch)
        optimizer_lstm.zero_grad()
        loss.backward()
        optimizer_lstm.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}")
'''

# 🔁 5. Entrenamiento
epochs = 10
train_losses_lstm = []
test_losses_lstm = []
train_maes_lstm = []
test_maes_lstm = []

for epoch in range(epochs):
    model_lstm.train()
    epoch_loss = 0
    train_predictions = []
    train_actuals = []
    for x_batch, y_batch in train_loader:
        output = model_lstm(x_batch)
        loss = criterion_lstm(output, y_batch)
        optimizer_lstm.zero_grad()
        loss.backward()
        optimizer_lstm.step()
        epoch_loss += loss.item()
        train_predictions.append(output.detach().numpy())
        train_actuals.append(y_batch.detach().numpy())

    train_losses_lstm.append(epoch_loss/len(train_loader))
    train_predictions = scaler_y.inverse_transform(np.vstack(train_predictions))
    train_actuals = scaler_y.inverse_transform(np.vstack(train_actuals))
    train_mae = mean_absolute_error(train_actuals, train_predictions)
    train_maes_lstm.append(train_mae)

    model_lstm.eval()
    test_loss = 0
    test_predictions = []
    test_actuals = []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            output = model_lstm(x_batch)
            loss = criterion_lstm(output, y_batch)
            test_loss += loss.item()
            test_predictions.append(output.numpy())
            test_actuals.append(y_batch.numpy())

    test_losses_lstm.append(test_loss/len(test_loader))
    test_predictions = scaler_y.inverse_transform(np.vstack(test_predictions))
    test_actuals = scaler_y.inverse_transform(np.vstack(test_actuals))
    test_mae = mean_absolute_error(test_actuals, test_predictions)
    test_maes_lstm.append(test_mae)


    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses_lstm[-1]:.4f}, Test Loss: {test_losses_lstm[-1]:.4f}, Train MAE: {train_maes_lstm[-1]:.4f}, Test MAE: {test_maes_lstm[-1]:.4f}")

# Plotting the training and validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_losses_lstm, label="Training Loss")
plt.plot(test_losses_lstm, label="Validation Loss")
plt.title("Training and Validation Loss per Epoch (LSTM)")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.grid(True)

# Plotting the training and validation MAE
plt.subplot(1, 2, 2)
plt.plot(train_maes_lstm, label="Training MAE")
plt.plot(test_maes_lstm, label="Validation MAE")
plt.title("Training and Validation MAE per Epoch (LSTM)")
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 📊 6. Evaluación
model_lstm.eval()
preds_lstm, actuals_lstm = [], []

with torch.no_grad():
    for x_batch, y_batch in test_loader:
        output = model_lstm(x_batch)
        preds_lstm.append(output.numpy())
        actuals_lstm.append(y_batch.numpy())

preds_lstm = scaler_y.inverse_transform(np.vstack(preds_lstm))
actuals_lstm = scaler_y.inverse_transform(np.vstack(actuals_lstm))

# 📈 7. Visualización
plt.figure(figsize=(12, 4))
plt.plot(actuals_lstm[:300], label="Real")
plt.plot(preds_lstm[:300], label="Predicho")
plt.title("Consumo energético - Real vs Predicho (LSTM)")
plt.xlabel("Tiempos")
plt.ylabel("kWh")
plt.legend()
plt.show()

# Calcular métricas
mae_lstm = mean_absolute_error(actuals_lstm, preds_lstm)
mse_lstm = mean_squared_error(actuals_lstm, preds_lstm)
r2_lstm = r2_score(actuals_lstm, preds_lstm)

print(f"LSTM MAE: {mae_lstm:.4f}")
print(f"LSTM MSE: {mse_lstm:.4f}")
print(f"LSTM R2 Score: {r2_lstm:.4f}")


In [ ]:
# Now save the trained models
torch.save(model_gru.state_dict(), 'energy_gru_model.pth')
torch.save(model_lstm.state_dict(), 'energy_lstm_model.pth')